In [40]:
from search import *

In [ ]:
import copy

import re

from search import *


class Movix(Problem):
    N = 0
    Y = 0

    def __init__(self, N, Y, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self.N = N
        self.Y = Y
        self._actions = []

        for i in range(1, N):
            self._actions.append(('V', (0, i)))
            self._actions.append(('V', (0, -i)))

        for i in range(1, N):
            self._actions.append(('H', (i, 0)))
            self._actions.append(('H', (-i, 0)))

        for i in range(-1, 2):
            for j in range(-1, 2):
                if i != 0 or j != 0:
                    self._actions.append(('L', (i, j)))
                    if i == 0 or j == 0:
                        self._actions.append(('S', (2 * i, 2 * j)))

    def actions(self, s):
        actions_list = []
        for a in self._actions:
            for pos in s[0][a[0]]:
                if self._is_valid(a, pos, s):
                    actions_list.append("Mover ficha {} de ({},{}) a ({},{})".format(a[0], pos[0], pos[1], (pos[0] + a[1][0]), (pos[1] + a[1][1])))

        return actions_list

    def _is_valid(self, a, pos, s):
        fil, col = (pos[0] + a[1][0], pos[1] + a[1][1])
         # Verificamos si la ficha está dentro de los límites del tablero
        if fil >= self.N or fil < 0 or col >= self.N or col < 0:
            return False  # Si alguna ficha está fuera de los límites, el estado no es válido
        if fil in s[1] and col in s[1][fil]:
            return False
        if a[0] == 'V' and fil in s[1] and (any(pos[1] < y < col for y in s[1][fil]) or any(col < y < pos[1] for y in s[1][fil])):
            return False
        if a[0] == 'H' and col in s[2] and (any(pos[0] < x < fil for x in s[2][col]) or any(fil < x < pos[0] for x in s[2][col])):
            return False
        if a[0] == 'S' and not (pos[0] + a[1][0]/2 in s[1] and (pos[1] + a[1][1]/2) in s[1][pos[0] + a[1][0]/2]):
            return False
        return True       # Si todas las fichas están dentro de los límites, y cumplen las propiedades el estado es válido

    def result(self, s, action):

        patron = r"Mover ficha (\w) de \((\d+),(\d+)\) a \((\d+),(\d+)\)"
        # Buscar los valores
        resultado = re.findall(patron, action)

        if resultado:
            ficha, x1, y1, x2, y2 = resultado[0]
            x1, y1 = int(x1), int(y1)
            x2, y2 = int(x2), int(y2)

            new_s = copy.deepcopy(s)

            new_s[0][ficha].remove((x1, y1))
            #no deberia hacer falta comprobar con estos ifs 
            if y1 in new_s[2] and x1 in new_s[2][y1]:
                new_s[2][y1].pop(x1)
            if x1 in new_s[1] and y1 in new_s[1][x1]:
                new_s[1][x1].pop(y1)

            
            if x1 in new_s[1] and len(new_s[1][x1]) == 0:
                del new_s[1][x1]
            if y1 in new_s[2] and len(new_s[2][y1]) == 0:
                del new_s[2][y1]

            
            if x2 not in new_s[1]:
                new_s[1][x2] = {}

            if y2 not in new_s[2]:
                new_s[2][y2] = {}

            new_s[0][ficha].append((x2, y2))
            new_s[1][x2][y2] = ficha
            new_s[2][y2][x2] = ficha

            #nos aseguramos q todo este ordenado
            
            #return new_s
            return (new_s[0], k: my_dict[k] for k in sorted(new_s[1]), k: my_dict[k] for k in sorted(new_s[2]))
        else:
            return None

    def goal_test(self, s):
        #linea = (len(s[1]) == 1 and max(s[2].keys()) - min(s[2].keys()) == self.Y*4 - 1) or (len(s[2]) == 1 and max(s[1].keys()) - min(s[1].keys()) == self.Y*4 - 1)
        linea = len(s[1]) == 1 or len(s[2]) == 1

        diagonal = True
        #if len(s[1]) > 0 and len(s[1][min(s[1])]) > 0:
        clave_minima = min(s[1])
        diferencia = clave_minima - min(s[1][clave_minima])
        for i in s[1]:
            if(len(s[1][i]) > 1):
                diagonal = False
                break
            elif(i - min(s[1][i]) != diferencia):
                diagonal = False
                break

        return linea or diagonal


In [ ]:
s = (
    {'S': [(1, 1)], 'H': [(3, 1)], 'L': [(2, 2)], 'V': [(0, 3)]},
    {0: {3: 'V'}, 1: {1: 'S'}, 2: {2: 'L'}, 3: {1: 'H'}},
    {1: {1: 'S', 3: 'H'}, 2: {2: 'L'}, 3: {0: 'V'}}
    )

#(('S', (1, 1)), ('H', (3, 1)), ('L', (2, 2)), ('V', (0, 3))) #(('_', '_', '_', '_', '_'), ('_', 'S', '_', 'H', '_'), ('_', '_', 'L', '_', '_'), ('V', '_', '_', '_', '_'), ('_', '_', '_', '_', '_')) #{'V':(0, 0), 'H':(3, 3), 'S':(2, 2), 'L':(4, 4)}

In [ ]:
p = Movix(5, 1, s)
p.initial

In [ ]:
p.actions(p.initial)

In [ ]:
breadth_first_tree_search(Movix(10, 1, s)).solution()

In [ ]:
s = (
    {'S': [(1, 7), (3, 7)], 'H': [(1, 5), (8, 2)], 'L': [(4, 1), (3, 9)], 'V': [(4, 6), (6, 7)]},
    {1: {5: 'H', 7: 'S'}, 3: {7: 'S', 9: 'L'}, 4: {1: 'L', 6: 'V'}, 6: {7: 'V'}, 8: {2: 'H'}},
    {1: {6: 'L'}, 2: {8: 'H'}, 5: {1: 'H'}, 6: {4: 'V'}, 7: {1: 'S', 3: 'S', 6: 'V'}, 9: {3: 'L'}}
)
p = Movix(10, 2, s)

In [ ]:
breadth_first_tree_search(p).solution()